In [2]:
import numpy as np
import cv2
import pandas as pd
from pathlib import Path
from numpy import expand_dims
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
from time import time
from time import sleep

import multiprocessing as mp
import psutil

import concurrent.futures


Using TensorFlow backend.
/home/base/anaconda3/envs/celeb/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:469: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/base/anaconda3/envs/celeb/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:470: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/base/anaconda3/envs/celeb/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:471: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/base/anaconda3/envs/cel

In [3]:
face_cascade = cv2.CascadeClassifier(str(Path.cwd() / 'lbpcascade_frontalface_improved.xml'))
brt = 90  # value could be + or - for brightness or darkness
gray=False
p=35# frame size around detected face
width=height=224 # size of the cropped image. Same as required for network
mitte=np.empty(shape=[0, 0])
mittleres_Gesicht_X=()

resnet50_features = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3),
                                pooling='avg')  # pooling: None, avg or max

EMBEDDINGS_Celebs=pd.read_json(Path.cwd() / 'EMBEDDINGS_8k.json')


In [4]:
def splitDataFrameIntoSmaller(df, chunkSize):
    listOfDf = list()
    numberChunks = len(df) // chunkSize + 1
    for i in range(numberChunks):
        listOfDf.append(df[i*chunkSize:(i+1)*chunkSize])
    return listOfDf
      
def faceembedding(Daten):
    Dist=[]
    for i in range(len(Daten.File)):
        Celebs=np.array(Daten.Embedding[i]) 
        Dist.append(np.linalg.norm(EMBEDDINGS-Celebs))    
    return Dist

celeb_embeddings=splitDataFrameIntoSmaller(EMBEDDINGS_Celebs, int(np.ceil(len(EMBEDDINGS_Celebs)/4)))   
g


In [13]:
aber=np.array(celeb_embeddings[0].Embedding)
aber[0]

[[0.5819720626,
  2.5359799862,
  0.0,
  4.6416273117,
  1.4086396694,
  1.7531970739,
  0.0,
  1.7581751347,
  0.1057061255,
  0.0,
  0.5051655173,
  0.9828417897,
  6.8900790215,
  0.1789514422,
  1.7093180418,
  0.0,
  0.6938746572000001,
  0.39155104760000004,
  1.9597799778,
  0.0,
  2.266122818,
  2.9472420216,
  0.0,
  0.0,
  3.2288198471,
  0.2746618688,
  0.0604142882,
  1.1160287857,
  1.4280257225000001,
  0.4260148406,
  0.594438076,
  0.0,
  0.9324651361,
  0.0,
  0.0,
  1.8121435642,
  0.8161889911,
  0.0,
  0.0,
  0.0,
  0.2848279476,
  0.6632480025,
  0.3923328817,
  4.3683691025,
  6.0305280685,
  0.0,
  1.9773098230000001,
  0.0,
  0.2537782192,
  1.1791149378,
  0.0,
  0.3161715567,
  0.3126646578,
  0.0839986056,
  0.0,
  2.1247527599,
  0.07309022550000001,
  0.028074055900000002,
  0.0,
  0.6312502027,
  0.3015416265,
  0.0,
  0.262406379,
  0.0,
  2.2449593544,
  0.0,
  0.0,
  1.7385935783000002,
  0.0,
  2.2654600143,
  0.0,
  7.4287776947,
  0.05236562340000000

In [4]:
cap = cv2.VideoCapture(0)
#cap.release()
ret, frame = cap.read() 
framemitte=np.shape(frame)[1]/2

In [5]:
while(True):
# CAPTURE FRAME BY FRAME
    ret, frame = cap.read() 
    frame=cv2.flip(frame,1)   
    cv2.imshow('frame', frame) 
    
#DECTECT FACE IN VIDEO CONTINUOUSLY       
    faces_detected = face_cascade.detectMultiScale(frame, scaleFactor=1.2, minNeighbors=5)#, Size(50,50))
    for (x,y,w,h) in faces_detected:
        rechteck=cv2.rectangle(frame, (x-p, y-p+2), (x+w+p, y+h+p+2), (0, 255, 0), 2)  
        cv2.imshow('frame', rechteck)     

# DETECT KEY INPUT  - ESC OR FIND MOST CENTERED FACE  
    key = cv2.waitKey(1)
    if key == 27: #Esc key
        cap.release()
        cv2.destroyAllWindows()
        break
    if key ==32: 
        mittleres_Gesicht_X=()
        mitte=()
        if faces_detected != (): # only if the cascader detected a face, otherwise error
            start1 = time()
#FIND MOST MIDDLE FACE            
            for (x,y,w,h) in faces_detected:
                mitte=np.append(mitte,(x+w/2))               
            mittleres_Gesicht_X = (np.abs(mitte - framemitte)).argmin()
            end1 = time()
            print('detect middel face ', end1-start1)
# FRAME THE DETECTED FACE
            start2=time()
            print(faces_detected[mittleres_Gesicht_X])
            (x, y, w, h) = faces_detected[mittleres_Gesicht_X]
            img=frame[y-p+2:y+h+p-2, x-p+2:x+w+p-2] #use only the detected face; crop it +2 to remove frame # CHECK IF IMAGE EMPTY (OUT OF IMAGE = EMPTY)     

            if len(img) != 0: # Check if face is out of the frame, then img=[], throwing error
                end2=time()
                print('detect face ',end2-start2)
# CROP IMAGE 
                start3=time()
                if img.shape > (width,height): #downsampling
                    img_small=cv2.resize(img, (width, height), interpolation=cv2.INTER_AREA) #resize the image to desired dimensions e.g., 256x256  
                elif img.shape < (width,height): #upsampling
                    img_small=cv2.resize(img, (width, height), interpolation=cv2.INTER_CUBIC) #resize the image to desired dimensions e.g., 256x256                      
                cv2.imshow('frame',img_small)
                cv2.waitKey(1) #hit any key
                end3=time()
                print('face crop', end3-start3)
#CREATE FACE EMBEDDINGS
                start4=time()
                pixels = img_small.astype('float32')
                samples = expand_dims(pixels, axis=0)
                samples = preprocess_input(samples, version=2)
                EMBEDDINGS = resnet50_features.predict(samples)
                end4=time()
                print('create face embeddings' , end4-start4)
# READ CELEB EMBEDDINGS AND COMPARE  
                start5=time()
                EuDist=[]
                with concurrent.futures.ThreadPoolExecutor(max_workers=1) as executor:
#                    ergebniss_1=executor.submit(faceembedding,Teil1)
#                    ergebniss_2=executor.submit(faceembedding,Teil2)
#                    ergebniss_3=executor.submit(faceembedding,Teil3)
#                    ergebniss_4=executor.submit(faceembedding,Teil4)
                    
                    ergebniss_1=executor.submit(faceembedding,celeb_embeddings[0])
                    ergebniss_2=executor.submit(faceembedding,celeb_embeddings[1])
                    ergebniss_3=executor.submit(faceembedding,celeb_embeddings[2])
                    ergebniss_4=executor.submit(faceembedding,celeb_embeddings[3])

                if ergebniss_1.done() and ergebniss_2.done() and ergebniss_3.done() and ergebniss_4.done():
                    EuDist.extend(ergebniss_1.result())
                    EuDist.extend(ergebniss_2.result())
                    EuDist.extend(ergebniss_3.result())
                    EuDist.extend(ergebniss_4.result())

/home/base/anaconda3/envs/celeb/lib/python3.6/site-packages/ipykernel_launcher.py:22: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


detect middel face  8.988380432128906e-05
[143 175 104 104]
detect face  0.00014591217041015625
face crop 0.004397869110107422
create face embeddings 0.32410335540771484


KeyError: 0

In [ ]:
cap.release()